In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('image', cmap='gray')
%matplotlib ipympl

import numpy as np
import pandas as pd
from PIL import Image, ImageDraw
import cv2
from tqdm import tqdm
import random

from stardist import _draw_polygons
from stardist.models import StarDist2D
from csbdeep.utils import normalize
import skimage

from scipy.optimize import dual_annealing, minimize

from tifffile import imsave, imread

2024-02-14 16:44:07.020260: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
# stardist model trained for 50 epochs on synthetic dataset
#model_choice = 'synthetic'
# stardist model trained on hough-circle dataset     
#model_choice = 'hough' 
# stardist model trained for 50 epochs on synthetic dataset starting from the pretrained 2D versatile fluo model
model_choice = 'synthetic_modified_2D_versatile_fluo' 

if model_choice == 'hough':
    model = StarDist2D(None, name = 'stardist_1707523110.807234', basedir = './models/')
elif model_choice == 'synthetic_modified_2D_versatile_fluo':
    model = StarDist2D(None, name = 'modified_2D_versatile_fluo', basedir = './models/')
elif model_choice == 'synthetic':
    model = StarDist2D(None, name = 'stardist_synthetic_1707847742.498359', basedir = './models/')
print(model)

Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.988092, nms_thresh=0.3.
StarDist2D(modified_2D_versatile_fluo): YXC → YXC
├─ Directory: /Users/matteoscandola/MasterThesis/tracking/models/modified_2D_versatile_fluo
└─ Config2D(n_dim=2, axes='YXC', n_channel_in=1, n_channel_out=33, train_checkpoint='weights_best.h5', train_checkpoint_last='weights_last.h5', train_checkpoint_epoch='weights_now.h5', n_rays=32, grid=(2, 2), backbone='unet', n_classes=None, unet_n_depth=3, unet_kernel_size=[3, 3], unet_n_filter_base=32, unet_n_conv_per_depth=2, unet_pool=[2, 2], unet_activation='relu', unet_last_activation='relu', unet_batch_norm=False, unet_dropout=0.0, unet_prefix='', net_conv_after_unet=128, net_input_shape=[None, None, 1], net_mask_shape=[None, None, 1], train_shape_completion=False, train_completion_crop=32, train_patch_size=[256, 256], train_background_reg=0.0001, train_foreground_only=0.9, train_sample_cach

In [11]:
def get_frame_sharp(cap, frame, x1, y1, x2, y2, w, h, preprocess):
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame)
    ret, image = cap.read()
    if preprocess:
        npImage = np.array(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))
        alpha = Image.new('L', (w, h), 0)
        draw = ImageDraw.Draw(alpha)
        draw.pieslice(((x1, y1), (x2, y2)), 0, 360, fill=255)
        npAlpha = np.array(alpha)
        npImage = npImage*npAlpha
        ind = np.where(npImage == 0)
        npImage[ind] = npImage[200, 200]
        kernel = np.array([[0, -1, 0],
                           [-1, 5,-1],
                           [0, -1, 0]])
        # sharpen image https://en.wikipedia.org/wiki/Kernel_(image_processing)
        npImage = cv2.filter2D(src=npImage, ddepth=-1, kernel=2*kernel)
        npImage = npImage[y1:y2, x1:x2]
        return normalize(npImage)
    elif not preprocess:
        return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    else:
        raise ValueError("preprocess must be a boolean")
    

def get_frame(cap, frame, x1, y1, x2, y2, w, h):
	cap.set(cv2.CAP_PROP_POS_FRAMES, frame)
	ret, image = cap.read()
	npImage = np.array(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))
	alpha = Image.new('L', (w, h), 0)
	draw = ImageDraw.Draw(alpha)
	draw.pieslice(((x1, y1), (x2, y2)), 0, 360, fill=255)
	npAlpha = np.array(alpha)
	npImage = npImage*npAlpha
	ind = np.where(npImage == 0)
	npImage[ind] = npImage[200, 200]
	npImage = npImage[y1:y2, x1:x2]
	return cv2.resize(npImage, (500, 500))

In [12]:
video_selection = "25b-25r"
#video_selection = "49b-1r"

system_name     = f"{video_selection} system"

if video_selection == "25b-25r":
    source_path     = "./data/25b25r-1.mp4" 
    xmin, ymin, xmax, ymax = 95, 30, 535, 470
    nDrops = 50
    save_path = './25b_25r/synthetic_trained/'

elif video_selection == "49b-1r":
    source_path     = "./data/49b1r.mp4"
    xmin, ymin, xmax, ymax = 20, 50, 900, 930
    nDrops = 50
    save_path = './49b_1r/synthetic_trained/'

video = cv2.VideoCapture(source_path)
video.set(cv2.CAP_PROP_POS_FRAMES, 0)
w = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video.get(cv2.CAP_PROP_FPS))
n_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"Video has {n_frames} frames with a resolution of {w}x{h} and a framerate of {fps} fps")
_, first_frame = video.read()
first_frame = first_frame[ymin:ymax, xmin:xmax]

preprocessed_img = get_frame(video, 0, xmin, ymin, xmax, ymax, w, h)
fig, (ax, ax1) = plt.subplots(1, 2, figsize = (10, 6), sharex=True, sharey=True)
ax.imshow(cv2.resize(cv2.cvtColor(first_frame, cv2.COLOR_BGR2RGB), (500, 500)))
ax.set(title = "Original video at frame 0", xlabel = "X [px]", ylabel = "Y [px]")
ax1.imshow(preprocessed_img)
ax1.set(title = "Preprocessed video at frame 0", xlabel = "X [px]", ylabel = "Y [px]")
plt.tight_layout()
plt.savefig(save_path + f"preprocessed_frame_0_{model_choice}.pdf", format = "pdf")
plt.close()

Video has 540000 frames with a resolution of 640x480 and a framerate of 30 fps


In [ ]:
selected_frame = n_frames-1
img = get_frame(video, selected_frame , xmin, ymin, xmax, ymax, w, h)
segmented_image, dict_test = model.predict_instances(normalize(img), predict_kwargs = {'verbose' : False})
n_feature_detected = len(dict_test['prob'])

fig, (ax, ax1) = plt.subplots(1, 2, figsize = (10, 6), sharex=True, sharey=True)
ax.imshow(img)
ax.set(title = 'Preprocessed Image', xlabel='X [px]', ylabel='Y [px]')
ax1.imshow(segmented_image)
ax1.set(title = f"Stardist result", xlabel='X [px]', ylabel='Y [px]')
plt.suptitle(f"Detection at frame {selected_frame}-- {n_feature_detected} features detected")
plt.tight_layout()
plt.savefig(save_path + f'stardist_example_{model_choice}.pdf', format='pdf')
plt.close()


fig, (ax, ax1) = plt.subplots(1, 2, figsize = (10, 6), sharex=True, sharey=True)
coord, points, prob = dict_test['coord'], dict_test['points'], dict_test['prob']
ax.imshow(img)
ax.set(title = 'Preprocessed Image', xlabel='X [px]', ylabel='Y [px]')
ax1.imshow(img)
_draw_polygons(coord, points, prob, show_dist=True)
ax1.set(title = f"Stardist result at frame {selected_frame} -- {n_feature_detected} features detected", xlabel='X [px]', ylabel='Y [px]')
plt.tight_layout()
plt.savefig(save_path + f'stardist_example2_{model_choice}.pdf', format='pdf')
plt.close()

No such comm: d5d350d5f8c145ddac323b7802358ed1
No such comm: d5d350d5f8c145ddac323b7802358ed1
No such comm: d5d350d5f8c145ddac323b7802358ed1
No such comm: d5d350d5f8c145ddac323b7802358ed1
No such comm: d5d350d5f8c145ddac323b7802358ed1
No such comm: d5d350d5f8c145ddac323b7802358ed1
No such comm: 1d7601502eb14126bd5eaaf9fd19beac
No such comm: 1d7601502eb14126bd5eaaf9fd19beac
No such comm: 1d7601502eb14126bd5eaaf9fd19beac
No such comm: 1d7601502eb14126bd5eaaf9fd19beac
No such comm: 1d7601502eb14126bd5eaaf9fd19beac
No such comm: 1d7601502eb14126bd5eaaf9fd19beac


In [ ]:
def detect_features(sample_frames):
    feature_properties_dict = {'frame':[], 'centroid-1':[], 'centroid-0':[], 'area':[], 'r':[], 'eccentricity':[],\
                                'prob':[], 'area_bbox':[], 'area_convex':[], 'area_filled':[], 'axis_major_length':[],\
                                'axis_minor_length':[], 'bbox-0':[], 'bbox-1':[], 'bbox-2':[], 'bbox-3':[],\
                                'equivalent_diameter_area':[], 'euler_number':[], 'extent':[], 'feret_diameter_max':[],\
                                'inertia_tensor-0-0':[], 'inertia_tensor-0-1':[], 'inertia_tensor-1-0':[],\
                                'inertia_tensor-1-1':[], 'inertia_tensor_eigvals-0':[], 'inertia_tensor_eigvals-1':[],\
                                'label':[]}
    for frame in tqdm(sample_frames):
        img = get_frame(video, frame, xmin, ymin, xmax, ymax, w, h)
        segmented_image, dict_test = model.predict_instances(normalize(img), predict_kwargs = {'verbose' : False})

        feature_properties = skimage.measure.regionprops_table(segmented_image, \
                                                                properties=('area', 'area_bbox', 'area_convex', 'area_filled',\
                                                                            'axis_major_length', 'axis_minor_length',\
                                                                            'bbox', 'centroid', 'eccentricity', \
                                                                            'equivalent_diameter_area', 'euler_number', 'extent',\
                                                                            'feret_diameter_max', 'inertia_tensor',\
                                                                            'inertia_tensor_eigvals', 'label'))

        for key in feature_properties.keys():
            feature_properties_dict[key] += list(feature_properties[key])
            
        feature_properties_dict['prob']  += list(dict_test['prob'])
        feature_properties_dict['frame'] += list(np.ones(len(list(feature_properties['centroid-0'])))*frame)
    # save data
    print("Saving data to dataframe...")
    feature_properties_dict['r'] = np.sqrt(np.array(feature_properties_dict['area'])/np.pi)
    raw_detection_df = pd.DataFrame(feature_properties_dict)
    raw_detection_df.rename(columns={'centroid-0': 'y', 'centroid-1': 'x'}, inplace=True)
    raw_detection_df['frame'] = raw_detection_df.frame.astype('int')
    raw_detection_df.sort_values(by=['frame', 'prob'], ascending=[True, False], inplace=True)
    raw_detection_df.to_parquet(save_path + f'raw_detection_{video_selection}_{model_choice}.parquet')
    return raw_detection_df

def filter_detection_data(r_min, r_max, raw_detection_df, nDrops):
    # filter found features
    print("Frames with spurious effects pre filtering:", len(np.where(raw_detection_df.groupby('frame').count().x.values != nDrops)[0]), "/", len(raw_detection_df.frame.unique()))

    filtered_df = raw_detection_df.loc[raw_detection_df.r.between(rmin, rmax)]
    filtered_df = filtered_df.groupby('frame').apply(lambda x: x.nlargest(nDrops, 'prob'))
    filtered_df = filtered_df.reset_index(drop=True)

    print("Frames with spurious effects after filtering:", len(np.where(filtered_df.groupby('frame').count().x.values != nDrops)[0]), "/", len(filtered_df.frame.unique()))

    return filtered_df

In [ ]:
if 0:
    startFrame = 0
    endFrame = 10000
    print(f"Processing from {int(startFrame/fps)} s to {int(endFrame/fps)} s")
    sample_frames = np.arange(startFrame, endFrame, 1, dtype=int)

#
sample_frames = np.sort(np.random.choice(np.arange(0, n_frames, 1, dtype=int), 1000, replace=False))
run_verb = True
save_verb = True
raw_detection_df = detect_features(sample_frames)

In [ ]:
# analyze the result of raw features location
n_feature_per_frame = raw_detection_df.groupby('frame').count().x.values

fig, ax = plt.subplots(2, 2, figsize = (8, 4))
ax[0, 0].plot(raw_detection_df.frame.unique(), n_feature_per_frame, '.')
ax[0, 0].set(xlabel = 'Frame', ylabel = 'N of droplets', title = 'N of droplets per frame')
ax[0, 1].plot(raw_detection_df.r, '.')
ax[0, 1].set(xlabel = 'Feature index', ylabel = 'Radius [px]', title = 'Radius of features detected')
ax[1, 0].scatter(raw_detection_df.r, raw_detection_df.eccentricity, s=0.1)
ax[1, 0].set(xlabel = 'Radius [px]', ylabel='Eccentricity', title='R-eccentricity correlation')
ax[1, 1].scatter(raw_detection_df.r, raw_detection_df.prob, s=0.1)
ax[1, 1].set(xlabel = 'Radius [px]', ylabel='Probability', title='R-Probability correlation')
plt.tight_layout()
plt.savefig(save_path + f'raw_features_{model_choice}.png', dpi = 500)
plt.close()

try:
    selected_frame = sample_frames[np.where(raw_detection_df.groupby('frame').count().x.values < nDrops)[0][0]]
except:
    selected_frame = sample_frames[np.where(raw_detection_df.groupby('frame').count().x.values != nDrops)[0][0]]

img = get_frame(video, selected_frame, xmin, ymin, xmax, ymax, w, h)
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.set_title(f"Example of spurious effect at frame {selected_frame}")
ax.imshow(img, cmap='gray')
for i in range(len(raw_detection_df.loc[raw_detection_df.frame == selected_frame])):
    ax.add_artist(plt.Circle((raw_detection_df.loc[raw_detection_df.frame == selected_frame].x.values[i], raw_detection_df.loc[raw_detection_df.frame == selected_frame].y.values[i]), \
                                raw_detection_df.loc[raw_detection_df.frame == selected_frame].r.values[i], color='r', fill=False))
plt.savefig(save_path + f'example_of_spurious_effect_{model_choice}.png', dpi = 500)
plt.close()

In [ ]:
rmax, rmin = 12.5, 6.3
filtered_df = filter_detection_data(rmin, rmax, raw_detection_df, nDrops)

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(8, 4))
ax[0, 0].plot(filtered_df.frame.unique(), filtered_df.groupby('frame').count().x.values, '.')
ax[0, 0].set(xlabel = 'Frame', ylabel = 'N of droplets', title = 'N of droplets per frame')
ax[0, 1].plot(filtered_df.r, '.')
ax[0, 1].set(xlabel = 'Feature index', ylabel = 'Radius [px]', title = 'Radius of features detected')
ax[1, 0].hist(filtered_df.r, bins=100, density=True)
ax[1, 0].set(xlabel = 'Radius [px]', ylabel='Density', title='Radius distribution')
ax[1, 1].scatter(filtered_df.r, filtered_df.prob, s=0.1)
ax[1, 1].set(xlabel = 'Radius [px]', ylabel='Probability', title='Probability distribution')
plt.tight_layout()
plt.savefig(save_path + 'filtered_features.png', dpi = 500)
plt.show()


test_frame = sample_frames[np.where(filtered_df.groupby('frame').count().x.values != nDrops)[0][0]]
img = get_frame(video, test_frame, xmin, ymin, xmax, ymax, w, h, True)

fig, (ax, ax1) = plt.subplots(1, 2, figsize=(10, 5), sharex=False, sharey=False)
ax.set(title = f"Filtered detection at frame {test_frame}", xticks=[], yticks=[])
ax.imshow(img, cmap='gray')
for i in range(len(raw_detection_df.loc[raw_detection_df.frame == test_frame])):
    ax.add_artist(plt.Circle((raw_detection_df.loc[raw_detection_df.frame == test_frame].x.values[i],\
                            raw_detection_df.loc[raw_detection_df.frame == test_frame].y.values[i]), \
                            raw_detection_df.loc[raw_detection_df.frame == test_frame].r.values[i], color='r', fill=True, alpha=0.5))
ax1.set(title = f"Filtered detection at frame {test_frame}", xticks=[], yticks=[])
ax1.imshow(img, cmap='gray')
for i in range(len(filtered_df.loc[filtered_df.frame == test_frame])):
    ax1.add_artist(plt.Circle((filtered_df.loc[filtered_df.frame == test_frame].x.values[i],\
                            filtered_df.loc[filtered_df.frame == test_frame].y.values[i]), \
                            filtered_df.loc[filtered_df.frame == test_frame].r.values[i], color='r', fill=True, alpha=0.5))
plt.suptitle(f"Filtered detection at a frame with spurious effects")
plt.show()

test_frame = filtered_df.loc[filtered_df.r == min(filtered_df.r)].frame.values[0]
img = get_frame(video, test_frame, xmin, ymin, xmax, ymax, w, h, True)

fig, (ax, ax1) = plt.subplots(1, 2, figsize=(10, 5), sharex=False, sharey=False)

ax.set(title = f"Filtered detection at frame {test_frame}", xticks=[], yticks=[])
ax.imshow(img, cmap='gray')
for i in range(len(raw_detection_df.loc[raw_detection_df.frame == test_frame])):
    ax.add_artist(plt.Circle((raw_detection_df.loc[raw_detection_df.frame == test_frame].x.values[i],\
                            raw_detection_df.loc[raw_detection_df.frame == test_frame].y.values[i]), \
                            raw_detection_df.loc[raw_detection_df.frame == test_frame].r.values[i], color='r', fill=True, alpha=0.5))
ax1.set(title = f"Filtered detection at frame {test_frame}", xticks=[], yticks=[])
ax1.imshow(img, cmap='gray')
for i in range(len(filtered_df.loc[filtered_df.frame == test_frame])):
    ax1.add_artist(plt.Circle((filtered_df.loc[filtered_df.frame == test_frame].x.values[i],\
                            filtered_df.loc[filtered_df.frame == test_frame].y.values[i]), \
                            filtered_df.loc[filtered_df.frame == test_frame].r.values[i], color='r', fill=True, alpha=0.5))
plt.suptitle(f"Filtered detection with minimum radius detected")
plt.show()

test_frame = filtered_df.loc[filtered_df.r == max(filtered_df.r)].frame.values[0]
img = get_frame(video, test_frame, xmin, ymin, xmax, ymax, w, h, True)

fig, (ax, ax1) = plt.subplots(1, 2, figsize=(10, 5), sharex=False, sharey=False)

ax.set(title = f"Filtered detection at frame {test_frame}", xticks=[], yticks=[])
ax.imshow(img, cmap='gray')
for i in range(len(raw_detection_df.loc[raw_detection_df.frame == test_frame])):
    ax.add_artist(plt.Circle((raw_detection_df.loc[raw_detection_df.frame == test_frame].x.values[i],\
                            raw_detection_df.loc[raw_detection_df.frame == test_frame].y.values[i]), \
                            raw_detection_df.loc[raw_detection_df.frame == test_frame].r.values[i], color='r', fill=True, alpha=0.5))
ax1.set(title = f"Filtered detection at frame {test_frame}", xticks=[], yticks=[])
ax1.imshow(img, cmap='gray')
for i in range(len(filtered_df.loc[filtered_df.frame == test_frame])):
    ax1.add_artist(plt.Circle((filtered_df.loc[filtered_df.frame == test_frame].x.values[i],\
                            filtered_df.loc[filtered_df.frame == test_frame].y.values[i]), \
                            filtered_df.loc[filtered_df.frame == test_frame].r.values[i], color='r', fill=True, alpha=0.5))
plt.suptitle(f"Filtered detection with maximum radius detected")
plt.show()